In [1]:
import sys
import numpy
import pprint
import time
import os
import h5py
import argparse as argparse
import json

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import History
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [3]:
#my code
import shared

In [4]:
numDays = 90

In [ ]:
#warning do not execute in jupyter
ap=argparse.ArgumentParser()
ap.add_argument("-d", "--days", default=2,
    help="How many days of data to include in learning")
args = vars(ap.parse_args())
numDays= float(args["days"])

In [5]:
print("Will be loading data for past {} days.".format(numDays))

df = shared.getData(numDays)
df = df.drop(['state'],axis=1)

Will be loading data for past 90 days.
now: 1514976551000
timeToSubtract: 7776000000
effective_time : 1507200551000


In [6]:
pprint.pprint(df.dtypes)
print("There are %d rows in the dataframe" % len(df))
pprint.pprint(df)

pprint.pprint(df.axes)
indexOfTemperature = df.axes[1].tolist().index("temperature")

day                 int64
extHumidity         int64
extTemp           float64
fanspeed            int64
hour                int64
loungeHumidity     object
loungeTemp         object
minute              int64
mode              float64
month               int64
pirstatus           int64
temperature         int64
dtype: object
There are 2914 rows in the dataframe
      day  extHumidity  extTemp  fanspeed  hour loungeHumidity loungeTemp  \
0       3           40     23.0         0    10          39.00      23.00   
1       3           40     23.0         0    10          39.00      23.00   
2       3           40     23.0         0    10          38.00      23.00   
3       3           43     23.0         0    10          38.00      23.00   
4       3           43     23.0         0    10          38.00      23.00   
5       3           43     23.0         0    10          38.00      23.00   
6       3           43     23.0         0    10          39.00      23.00   
7       3           4

In [7]:
if indexOfTemperature == (len(df.axes[1])-1):
    # the temperature/label column is at the end.. easy
    dataset = df.values[:,0:(indexOfTemperature)].astype(float)
    labels = df.values[:,indexOfTemperature].astype(int)

In [8]:
#debug print
pprint.pprint(dataset)
pprint.pprint(labels)

array([[  3.,  40.,  23., ...,   1.,   0.,   0.],
       [  3.,  40.,  23., ...,   1.,   0.,   0.],
       [  3.,  40.,  23., ...,   1.,   0.,   0.],
       ..., 
       [ 24.,  25.,  23., ...,   1.,  11.,   2.],
       [ 24.,  25.,  23., ...,   1.,  11.,   2.],
       [ 24.,  25.,  23., ...,   1.,  11.,   3.]])
array([30, 30, 30, ..., 30, 30, 30])


In [9]:
#convert to numpy array
#dataset = 

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [10]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_labels = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
one_hot_labels = np_utils.to_categorical(encoded_labels)

In [11]:
pprint.pprint(encoded_labels)
pprint.pprint(one_hot_labels)

array([13, 13, 13, ..., 13, 13, 13], dtype=int64)
array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])


In [12]:
input_dimension = dataset.shape[1]
output_dimension = one_hot_labels.shape[1]
print("input_dimension", input_dimension)
print("output_dimension ", output_dimension)
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(11, activation='relu', input_dim=input_dimension))
    model.add(Dense(units=output_dimension))
    model.add(Activation("softmax"))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

input_dimension 11
output_dimension  14


In [13]:
model = baseline_model()
history = History()

In [14]:
model.fit(dataset,one_hot_labels, epochs=50, batch_size=3, callbacks=[history])

Epoch 1/50
2914/2914 [==============================] - 3s 865us/step - loss: 7.9142 - acc: 0.3974
Epoch 2/50
2914/2914 [==============================] - 2s 738us/step - loss: 0.8498 - acc: 0.7889
Epoch 3/50
2914/2914 [==============================] - 2s 650us/step - loss: 0.7533 - acc: 0.7872
Epoch 4/50
2914/2914 [==============================] - 2s 655us/step - loss: 0.7102 - acc: 0.7938
Epoch 5/50
2914/2914 [==============================] - 2s 658us/step - loss: 0.6919 - acc: 0.7948
Epoch 6/50
2914/2914 [==============================] - 2s 789us/step - loss: 0.6780 - acc: 0.7938
Epoch 7/50
2914/2914 [==============================] - 3s 1ms/step - loss: 0.6636 - acc: 0.8003
Epoch 8/50
2914/2914 [==============================] - 2s 838us/step - loss: 0.6527 - acc: 0.8023
Epoch 9/50
2914/2914 [==============================] - 2s 679us/step - loss: 0.6384 - acc: 0.7986
Epoch 10/50
2914/2914 [==============================] - 2s 775us/step - loss: 0.6252 - acc: 0.7989
Epoch 11/50

In [15]:
final_loss = history.history['loss'][len(history.history['loss'])-1]
if final_loss >  1.0:
    sys.exit("Unsatisfactory loss ( >1.0) found. will not save model.")

In [16]:
#create new model directory
base_path = "assets/trained-models"
this_model_name = "temperature-model-%d" % time.time()

#ensure directory exists
this_model_base_path = "%s/%s" % (base_path, this_model_name)
os.makedirs(this_model_base_path)
print("Created path " + this_model_base_path)

this_model_json_path = ("{}/model.json".format(this_model_base_path))
this_model_weights_path = ("{}/model.h5".format(this_model_base_path))
this_model_labels_path = ("{}/labels.json".format(this_model_base_path))

print("Saving model to ", this_model_json_path)
print("Saving weights to ", this_model_weights_path)
print("Saving labels to ", this_model_labels_path)

Created path assets/trained-models/temperature-model-1514976693
Saving model to  assets/trained-models/temperature-model-1514976693/model.json
Saving weights to  assets/trained-models/temperature-model-1514976693/model.h5
Saving labels to  assets/trained-models/temperature-model-1514976693/labels.json


In [17]:
model_json = model.to_json()
with open(this_model_json_path, "w") as json_file:
    json_file.write(model_json)
model.save_weights(this_model_weights_path)

#make labels and save
classes = [str(i) for i in encoded_labels]
labels = [str(i) for i in labels]
label_map = dict(zip(classes,labels))
label_map_json = json.dumps(label_map)

with open(this_model_labels_path, "w") as json_file:
    json_file.write(label_map_json)


In [18]:
#update the root index with the path to the latest model
model_latest_index = "%s/temperature-model.index" % base_path
with open(model_latest_index,"w") as model_latest_index_file:
    model_latest_index_file.write(this_model_name)
    
print("Model successfully saved.")

Model successfully saved.
